# ACCESS-NRI Intake catalog for loading model output

This tutorial demonstrates how to use the ACCESS-NRI Intake catalog to load model or other output.

⚠️ **Membership to project `xp65` is required to access the ACCESS-NRI Intake catalog** ⚠️

This is a concise version of the longer [ACCESS-NRI Intake catalog documentation](https://access-nri-intake-catalog.readthedocs.io/) and related [COSIMA training workshop](https://github.com/ACCESS-Hive/cosima-training-workshop-2023/blob/main/Intake.ipynb). Users are encouraged to refer to these for more detail and demonstrations.

Requirements: The `conda/analysis3` module from `/g/data/xp65/public/modules`.

# Start a dask Client

This is not specific to using the ACCESS-NRI Intake catalog, but it's useful!

In [1]:
from os import environ
environ["PYTHONWARNINGS"] = "ignore"

In [2]:
from dask.distributed import Client

client = Client(threads_per_worker=1)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 7
Total threads: 7,Total memory: 32.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:40963,Workers: 0
Dashboard: /proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:41791,Total threads: 1
Dashboard: /proxy/32819/status,Memory: 4.57 GiB
Nanny: tcp://127.0.0.1:33125,


# Opening and searching the catalog

To use the ACCESS-NRI Intake catalog, we need to import `intake`

In [3]:
import intake

We can open the catalog as follows

In [4]:
catalog = intake.cat.access_nri

The returned object `catalog` is an instance of the ACCESS-NRI Intake catalog that we can use to find and load data.

Printing the `catalog` object will return a dataframe of experiments that you can browse:

In [5]:
catalog

,model,description,realm,frequency,variable
name,,,,,
01deg_jra55_ryf_Control,{ACCESS-OM2-01},"{0.1° ACCESS-OM2 repeat year forcing control run for the simulations performed in Huguenin et al. (2024, GRL)}","{ocean, seaIce}","{1mon, fx}","{ty_trans_rho, vvel_m, area_t, ULON, fsalt_ai_m, vatm_m, alvdr_ai_m, opening_m, time_bounds, tx_trans_rho, frazil_on_nrho, temp_global_ave, total_ocean_calving_heat, strairy_m, yt_ocean, sfc_hflux..."
01deg_jra55_ryf_ENFull,{ACCESS-OM2},"{0.1° ACCESS-OM2 El Níño run for the simulations performed in Huguenin et al. (2024, GRL)}","{ocean, seaIce}","{1mon, fx}","{ty_trans_rho, vvel_m, area_t, ULON, fsalt_ai_m, vatm_m, alvdr_ai_m, opening_m, time_bounds, tx_trans_rho, temp_global_ave, total_ocean_calving_heat, strairy_m, yt_ocean, tmask, temp_advection, ua..."
01deg_jra55_ryf_LNFull,{ACCESS-OM2},"{0.1° ACCESS-OM2 La Níña run for the simulations performed in Huguenin et al. (2024, GRL)}","{ocean, seaIce}","{1mon, fx}","{ty_trans_rho, vvel_m, area_t, ULON, fsalt_ai_m, vatm_m, alvdr_ai_m, opening_m, time_bounds, tx_trans_rho, temp_global_ave, total_ocean_calving_heat, strairy_m, yt_ocean, tmask, temp_advection, ua..."
01deg_jra55v13_ryf9091,{ACCESS-OM2-01},{0.1 degree ACCESS-OM2 global model configuration with JRA55-do v1.3 RYF9091 repeat year forcing (May 1990 to Apr 1991)},"{ocean, seaIce}","{3mon, fx, 1mon, 1day, 3hr}","{ty_trans_rho, vvel_m, area_t, ULON, passive_prydz, fsalt_ai_m, vatm_m, alvdr_ai_m, opening_m, time_bounds, tx_trans_rho, temp_global_ave, total_ocean_calving_heat, strairy_m, yt_ocean, tmask, wfi..."
01deg_jra55v13_ryf9091_easterlies_down10,{ACCESS-OM2-01},{0.1 degree ACCESS-OM2 global model configuration with JRA55-do v1.3 RYF9091 repeat year forcing (May 1990 to Apr 1991) and zonal/meridional wind speed around Antarctica decreased by 10%.},"{ocean, seaIce}","{1mon, fx, 1day}","{vvel_m, area_t, ULON, fsalt_ai_m, vatm_m, alvdr_ai_m, opening_m, time_bounds, tx_trans_rho, temp_global_ave, total_ocean_calving_heat, strairy_m, yt_ocean, tmask, wfimelt, uarea, frazil_m, streng..."
01deg_jra55v13_ryf9091_easterlies_up10,{ACCESS-OM2-01},{0.1 degree ACCESS-OM2 global model configuration with JRA55-do v1.3 RYF9091 repeat year forcing (May 1990 to Apr 1991) and zonal/meridional wind speed around Antarctica increased by 10%.},"{ocean, seaIce}","{1mon, fx, 1day}","{vvel_m, area_t, ULON, fsalt_ai_m, vatm_m, alvdr_ai_m, opening_m, time_bounds, tx_trans_rho, temp_global_ave, total_ocean_calving_heat, strairy_m, yt_ocean, tmask, wfimelt, uarea, frazil_m, streng..."
01deg_jra55v13_ryf9091_easterlies_up10_meridional,{ACCESS-OM2-01},{0.1 degree ACCESS-OM2 global model configuration with JRA55-do v1.3 RYF9091 repeat year forcing (May 1990 to Apr 1991) and meridional wind speed around Antarctica increased by 10%.},"{ocean, seaIce}","{1mon, fx, 1day}","{vvel_m, area_t, ULON, fsalt_ai_m, vatm_m, alvdr_ai_m, opening_m, time_bounds, tx_trans_rho, temp_global_ave, total_ocean_calving_heat, strairy_m, yt_ocean, tmask, wfimelt, uarea, frazil_m, streng..."
01deg_jra55v13_ryf9091_easterlies_up10_zonal,{ACCESS-OM2-01},{0.1 degree ACCESS-OM2 global model configuration with JRA55-do v1.3 RYF9091 repeat year forcing (May 1990 to Apr 1991) and zonal wind speed around Antarctica increased by 10%.},"{ocean, seaIce}","{1mon, fx, 1day}","{vvel_m, area_t, ULON, fsalt_ai_m, vatm_m, alvdr_ai_m, opening_m, time_bounds, tx_trans_rho, temp_global_ave, total_ocean_calving_heat, strairy_m, yt_ocean, tmask, wfimelt, uarea, frazil_m, streng..."
01deg_jra55v13_ryf9091_qian_wthmp,{ACCESS-OM2},"{Future perturbations with wind, thermal and meltwater forcing, branching off 01deg_jra55v13_ryf9091, as described in Li et al. 2023, https://www.nature.com/articles/s41586-023-05762-w}","{ocean, seaIce}","{1mon, fx}","{vvel_m, area_t, ULON, fsalt_ai_m, vatm_m, alvdr_ai_m, opening_m, time_bounds, tx_trans_rho, temp_global_ave, total_ocean_calving_heat, strairy_m, yt_ocean, tmask, wfimelt, temp_advection, uarea, ..."


You can also search based on the columns in this dataframe to find experiments that are relevant to you. For example, you might be interested in all ACCESS-OM2 experiments that have the variable `"surface_salt"` at daily frequency. There are 6 such experiments currently available through the catalog:

In [6]:
catalog.search(model="ACCESS-OM2", variable="surface_salt", frequency="1day")

,model,description,realm,frequency,variable
name,,,,,
025deg_era5_iaf,{ACCESS-OM2},{0.25 degree ACCESS-OM2 global model configuration with ERA5 interannual\nforcing (1980-2021)},{ocean},{1day},{surface_salt}
025deg_era5_ryf,{ACCESS-OM2},{0.25 degree ACCESS-OM2 global model configuration with ERA5 RYF9091 repeat\nyear forcing (May 1990 to Apr 1991)},{ocean},{1day},{surface_salt}
025deg_jra55_iaf_era5comparison,{ACCESS-OM2},{0.25 degree ACCESS-OM2 global model configuration with JRA55-do v1.5.0\ninterannual forcing (1980-2019)},{ocean},{1day},{surface_salt}
025deg_jra55_ryf_era5comparison,{ACCESS-OM2},{0.25 degree ACCESS-OM2 global model configuration with JRA55-do v1.4.0\nRYF9091 repeat year forcing (May 1990 to Apr 1991)},{ocean},{1day},{surface_salt}
1deg_era5_iaf,{ACCESS-OM2},{1 degree ACCESS-OM2 global model configuration with ERA5 interannual\nforcing (1960-2019)},{ocean},{1day},{surface_salt}
1deg_jra55_iaf_era5comparison,{ACCESS-OM2},{1 degree ACCESS-OM2 global model configuration with JRA55-do v1.4.0\ninterannual forcing (1960-2019)},{ocean},{1day},{surface_salt}


# Opening data

There are [multiple ways](https://access-nri-intake-catalog.readthedocs.io/en/latest/usage/quickstart.html#loading-intake-sources) to open data from the experiments in `catalog`. Here we'll demonstrate how to do this when you know the name of the experiment you are interested in, since this typical for COSIMA users.

For example, we can open monthly data for the `surface_salt` variable in the `01deg_jra55v13_ryf9091` experiment as follows:

In [7]:
experiment = "01deg_jra55v13_ryf9091"
variable = "surface_salt"

In [8]:
data_ic = catalog[experiment].search(
    variable=variable, 
    frequency="1mon"
).to_dask()

/g/data/xp65/public/apps/med_conda/envs/analysis3-25.09/lib/python3.11/site-packages/intake_esm/core.py:301: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  records = grouped.get_group(internal_key).to_dict(orient='records')


In [9]:
data_ic["surface_salt"]

<xarray.DataArray 'surface_salt' (time: 3360, yt_ocean: 2700, xt_ocean: 3600)> Size: 131GB
dask.array<concatenate, shape=(3360, 2700, 3600), dtype=float32, chunksize=(1, 675, 900), chunktype=numpy.ndarray>
Coordinates:
  * xt_ocean  (xt_ocean) float64 29kB -279.9 -279.8 -279.7 ... 79.75 79.85 79.95
  * yt_ocean  (yt_ocean) float64 22kB -81.11 -81.07 -81.02 ... 89.89 89.94 89.98
  * time      (time) object 27kB 1900-01-16 12:00:00 ... 2179-12-16 12:00:00
Attributes:
    long_name:      Practical Salinity
    units:          psu
    valid_range:    [-10. 100.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    standard_name:  sea_surface_salinity

# Some important facts

There are a few important facts in the ACCESS-NRI Intake catalog that users should be aware of.

## 1. The catalog returns `Datasets` (not `Dataarray`s)

This is because with the catalog you can load multiple variables into a single dataset with a single call (when these variables are in the same file). For example,

In [10]:
data_ic_multivar = catalog[experiment].search(
    variable=["surface_salt", "surface_temp"], 
    frequency="1mon"
).to_dask()

/g/data/xp65/public/apps/med_conda/envs/analysis3-25.09/lib/python3.11/site-packages/intake_esm/core.py:301: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  records = grouped.get_group(internal_key).to_dict(orient='records')


In [11]:
data_ic_multivar

<xarray.Dataset> Size: 261GB
Dimensions:       (time: 3360, yt_ocean: 2700, xt_ocean: 3600)
Coordinates:
  * xt_ocean      (xt_ocean) float64 29kB -279.9 -279.8 -279.7 ... 79.85 79.95
  * yt_ocean      (yt_ocean) float64 22kB -81.11 -81.07 -81.02 ... 89.94 89.98
  * time          (time) object 27kB 1900-01-16 12:00:00 ... 2179-12-16 12:00:00
Data variables:
    surface_temp  (time, yt_ocean, xt_ocean) float32 131GB dask.array<chunksize=(1, 675, 900), meta=np.ndarray>
    surface_salt  (time, yt_ocean, xt_ocean) float32 131GB dask.array<chunksize=(1, 675, 900), meta=np.ndarray>
Attributes:
    filename:                        ocean_month.nc
    title:                           ACCESS-OM2-01
    grid_type:                       mosaic
    grid_tile:                       1
    intake_esm_vars:                 ['surface_temp', 'surface_salt']
    intake_esm_attrs:filename:       ocean_month.nc
    intake_esm_attrs:file_id:        ocean.1mon.nv:2.xt_ocean:3600.xu_ocean:3...
    intake_esm_attrs:frequency:      1mon
    intake_esm_attrs:realm:          ocean
    intake_esm_attrs:_data_format_:  netcdf
    intake_esm_dataset_key:          ocean.1mon.nv:2.xt_ocean:3600.xu_ocean:3...

## 2. The catalog knows which files make up distinct datasets

The catalog knows which files make up distinct datasets and provides methods to open multiple datasets from a single query. We can run the equivalent to the cell above using the catalog, using `to_dataset_dict()` rather than `to_dask()`. Doing so returns a dictionary containing Datasets of the variable at all the available frequencies (daily and monthly in this case).

In [12]:
data_ic_multifreq = catalog[experiment].search(variable=variable).to_dataset_dict()


--> The keys in the returned dictionary of datasets are constructed as follows:
	'file_id'


/g/data/xp65/public/apps/med_conda/envs/analysis3-25.09/lib/python3.11/site-packages/intake_esm/core.py:301: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  records = grouped.get_group(internal_key).to_dict(orient='records')
/g/data/xp65/public/apps/med_conda/envs/analysis3-25.09/lib/python3.11/site-packages/intake_esm/core.py:301: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  records = grouped.get_group(internal_key).to_dict(orient='records')
/g/data/xp65/public/apps/med_conda/envs/analysis3-25.09/lib/python3.11/site-packages/intake_esm/core.py:301: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. 

In [13]:
data_ic_multifreq

{'ocean.1day.nv:2.xt_ocean:3600.xu_ocean:3600.yt_ocean:2700.yu_ocean:2700': <xarray.Dataset> Size: 610GB
 Dimensions:       (time: 15695, yt_ocean: 2700, xt_ocean: 3600)
 Coordinates:
   * xt_ocean      (xt_ocean) float64 29kB -279.9 -279.8 -279.7 ... 79.85 79.95
   * yt_ocean      (yt_ocean) float64 22kB -81.11 -81.07 -81.02 ... 89.94 89.98
   * time          (time) object 126kB 2137-01-01 12:00:00 ... 2179-12-31 12:0...
 Data variables:
     surface_salt  (time, yt_ocean, xt_ocean) float32 610GB dask.array<chunksize=(1, 675, 900), meta=np.ndarray>
 Attributes:
     filename:                        ocean_daily.nc
     title:                           ACCESS-OM2-01
     grid_type:                       mosaic
     grid_tile:                       1
     intake_esm_vars:                 ['surface_salt']
     intake_esm_attrs:filename:       ocean_daily.nc
     intake_esm_attrs:file_id:        ocean.1day.nv:2.xt_ocean:3600.xu_ocean:3...
     intake_esm_attrs:frequency:      1day
     int

Alternatively, multiple datasets can be opened directly into an [xarray-datatree](https://xarray-datatree.readthedocs.io/en/latest/) by calling `to_datatree` rather than `to_dataset_dict` (in an upcoming release, it will be easier for users to control how the groups are structured in the datatree.). For example:

In [14]:
data_ic_datatree = catalog[experiment].search(variable=variable).to_datatree()


--> The keys in the returned dictionary of datasets are constructed as follows:
	'file_id'


/g/data/xp65/public/apps/med_conda/envs/analysis3-25.09/lib/python3.11/site-packages/intake_esm/core.py:301: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  records = grouped.get_group(internal_key).to_dict(orient='records')
/g/data/xp65/public/apps/med_conda/envs/analysis3-25.09/lib/python3.11/site-packages/intake_esm/core.py:301: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  records = grouped.get_group(internal_key).to_dict(orient='records')
/g/data/xp65/public/apps/med_conda/envs/analysis3-25.09/lib/python3.11/site-packages/intake_esm/core.py:301: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. 

In [15]:
data_ic_datatree

<xarray.DataTree>
Group: /
├── Group: /ocean.1day.nv:2.xt_ocean:3600.xu_ocean:3600.yt_ocean:2700.yu_ocean:2700
│       Dimensions:       (time: 15695, yt_ocean: 2700, xt_ocean: 3600)
│       Coordinates:
│         * xt_ocean      (xt_ocean) float64 29kB -279.9 -279.8 -279.7 ... 79.85 79.95
│         * yt_ocean      (yt_ocean) float64 22kB -81.11 -81.07 -81.02 ... 89.94 89.98
│         * time          (time) object 126kB 2137-01-01 12:00:00 ... 2179-12-31 12:0...
│       Data variables:
│           surface_salt  (time, yt_ocean, xt_ocean) float32 610GB dask.array<chunksize=(1, 675, 900), meta=np.ndarray>
│       Attributes:
│           filename:                        ocean_daily.nc
│           title:                           ACCESS-OM2-01
│           grid_type:                       mosaic
│           grid_tile:                       1
│           intake_esm_vars:                 ['surface_salt']
│           intake_esm_attrs:filename:       ocean_daily.nc
│           intake_esm_attrs:file_id:        ocean.1day.nv:2.xt_ocean:3600.xu_ocean:3...
│           intake_esm_attrs:frequency:      1day
│           intake_esm_attrs:realm:          ocean
│           intake_esm_attrs:_data_format_:  netcdf
│           intake_esm_dataset_key:          ocean.1day.nv:2.xt_ocean:3600.xu_ocean:3...
├── Group: /ocean.1day.nv:2.xt_ocean:3600.yt_ocean:2700
│       Dimensions:       (time: 65885, yt_ocean: 2700, xt_ocean: 3600)
│       Coordinates:
│         * xt_ocean      (xt_ocean) float64 29kB -279.9 -279.8 -279.7 ... 79.85 79.95
│         * yt_ocean      (yt_ocean) float64 22kB -81.11 -81.07 -81.02 ... 89.94 89.98
│         * time          (time) object 527kB 1956-04-01 12:00:00 ... 2136-12-31 12:0...
│       Data variables:
│           surface_salt  (time, yt_ocean, xt_ocean) float32 3TB dask.array<chunksize=(1, 675, 900), meta=np.ndarray>
│       Attributes: (12/14)
│           filename:                                ocean_daily.nc
│           title:                                   ACCESS-OM2-01
│           grid_type:                               mosaic
│           grid_tile:                               1
│           intake_esm_vars:                         ['surface_salt']
│           intake_esm_attrs:filename:               ocean_daily.nc
│           ...                                      ...
│           intake_esm_attrs:variable:               average_DT,average_T1,average_T2...
│           intake_esm_attrs:variable_long_name:     Length of average period,Start t...
│           intake_esm_attrs:variable_cell_methods:  ,,,time: mean,time: mean,time: m...
│           intake_esm_attrs:realm:                  ocean
│           intake_esm_attrs:_data_format_:          netcdf
│           intake_esm_dataset_key:                  ocean.1day.nv:2.xt_ocean:3600.yt...
└── Group: /ocean.1mon.nv:2.xt_ocean:3600.xu_ocean:3600.yt_ocean:2700.yu_ocean:2700
        Dimensions:       (time: 3360, yt_ocean: 2700, xt_ocean: 3600)
        Coordinates:
          * xt_ocean      (xt_ocean) float64 29kB -279.9 -279.8 -279.7 ... 79.85 79.95
          * yt_ocean      (yt_ocean) float64 22kB -81.11 -81.07 -81.02 ... 89.94 89.98
          * time          (time) object 27kB 1900-01-16 12:00:00 ... 2179-12-16 12:00:00
        Data variables:
            surface_salt  (time, yt_ocean, xt_ocean) float32 131GB dask.array<chunksize=(1, 675, 900), meta=np.ndarray>
        Attributes:
            filename:                        ocean_month.nc
            title:                           ACCESS-OM2-01
            grid_type:                       mosaic
            grid_tile:                       1
            intake_esm_vars:                 ['surface_salt']
            intake_esm_attrs:filename:       ocean_month.nc
            intake_esm_attrs:file_id:        ocean.1mon.nv:2.xt_ocean:3600.xu_ocean:3...
            intake_esm_attrs:frequency:      1mon
            intake_esm_attrs:realm:          ocean
            intake_esm_attrs:_data_format_:  netcdf
           

## 3. The frequency vocabulary:

In the catalog, frequency follows a standard vocabulary that is very similar to CMIP6:

```python
"fx" # fixed
"subhr" # subhourly
"<int>hr" # hourly
"<int>day" # daily
"<int>mon" # monthly
"<int>yr" # yearly
"<int>dec" # decadal
```

## 4. Method for passing keyword arguments

With the catalog, keyword argments for xarray's `open_dataset` and `combine_by_coords` functions are passed separately to `to_dask` (or `to_dataset_dict`). For example:

In [16]:
xarray_open_kwargs=dict(
    chunks={"xt_ocean": -1, "yt_ocean": -1}
)
xarray_combine_by_coords_kwargs=dict(
    compat="override",
    data_vars="minimal",
    coords="minimal"
)

data_ic_kw = catalog[experiment].search(
    variable=variable, 
    frequency="1mon"
).to_dask(
    xarray_open_kwargs=xarray_open_kwargs,
    xarray_combine_by_coords_kwargs=xarray_combine_by_coords_kwargs,
)

/g/data/xp65/public/apps/med_conda/envs/analysis3-25.09/lib/python3.11/site-packages/intake_esm/core.py:301: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  records = grouped.get_group(internal_key).to_dict(orient='records')


## 5. Catalog does not allow search by start and end date

It's not possible to query on a time range with the Intake catalog.

We can always slice the time axis afterwards though. That is, with the catalog you'd just do:

In [17]:
data_ic = catalog[experiment].search(
    variable=variable, 
    frequency="1mon"
).to_dask()

start_time = "2000-01-01"
end_time = "2180-01-01"

data_ic_times = data_ic.sel(time=slice(start_time, end_time))

/g/data/xp65/public/apps/med_conda/envs/analysis3-25.09/lib/python3.11/site-packages/intake_esm/core.py:301: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  records = grouped.get_group(internal_key).to_dict(orient='records')


which takes a few seconds longer.

This difference is acceptable because the opening of datasets is a parallelized task that is done [lazily](https://docs.xarray.dev/en/stable/user-guide/dask.html#parallel-computing-with-dask),  so opening all files and reducing the times using xarray's `sel` methods doesn't add too much overhead. In most cases where the overhead of opening the files seems large, this can be reduced through sensible choices of keyword arguments provided to `open_dataset` and `combine_by_coords` - see the xarray documentation on [Reading multi-file datasets](https://docs.xarray.dev/en/stable/user-guide/io.html#reading-multi-file-datasets) for details.

# Tips, gotchas and workarounds

## 1. Speeding up opening your datasets

Try passing the following argument to your `to_dask` or `to_dataset_dict` call:

```python
xarray_combine_by_coords_kwargs=dict(
    compat="override",
    data_vars="minimal",
    coords="minimal"
)
```

See the xarray documentation on [Reading multi-file datasets](https://docs.xarray.dev/en/stable/user-guide/io.html#reading-multi-file-datasets) for more details about these arguments.

## 2. Choosing chunksizes

Correctly choosing chunk sizes when you open datasets will greatly improve the speed of your analysis. Check out the [Chunking tutorial](https://access-nri-intake-catalog.readthedocs.io/en/latest/usage/chunking.html) in the ACCESS-NRI Intake catalog documentation

## 3. Loading time-invariant variables

Many COSIMA experiments include multiple repeated files containing the same fixed frequency data (e.g. grid information). The catalog fails to concatenate these files since they don't contain clear dimension to concatenate along.

The workaround to the above is to further restrict your search to only return one file. For example, you could only return the file in the `output000` directory with:

In [18]:
data_ic_fixed = catalog[experiment].search(
    variable='area_t',
    # path=".*output000.*"
).to_dask()

/g/data/xp65/public/apps/med_conda/envs/analysis3-25.09/lib/python3.11/site-packages/intake_esm/core.py:301: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  records = grouped.get_group(internal_key).to_dict(orient='records')
/g/data/xp65/public/apps/med_conda/envs/analysis3-25.09/lib/python3.11/site-packages/intake_esm/source.py:306: ConcatenationWarning: Attempting to concatenate datasets without valid dimension coordinates: retaining only first dataset. Request valid dimension coordinate to silence this warning.
  warnings.warn(


## 4. Determining what can be searched upon in an experiment

You can see what can be `search`ed on within an experiment with:

In [19]:
catalog[experiment].df.columns.tolist()

['filename',
 'path',
 'file_id',
 'frequency',
 'start_date',
 'end_date',
 'variable',
 'variable_long_name',
 'variable_standard_name',
 'variable_cell_methods',
 'variable_units',
 'realm']

It can also be helpful sometimes to look at the `catalog[experiment].df` object itself, which is a dataframe of all of the files in the experiment and their metadata

In [21]:
catalog[experiment].df.head()

,filename,path,file_id,frequency,start_date,end_date,variable,variable_long_name,variable_standard_name,variable_cell_methods,variable_units,realm
0,iceh.1900-01.nc,/g/data/cj50/access-om2/raw-output/access-om2-01/01deg_jra55v13_ryf9091/output000/ice/OUTPUT/iceh.1900-01.nc,seaIce.1mon.d2:2.nc:5.ni:3600.nj:2700,1mon,"1900-01-01, 00:00:00","1900-02-01, 00:00:00",['ANGLE' 'ANGLET' 'HTE' 'HTN' 'NCAT' 'TLAT' 'TLON' 'Tsfc_m' 'ULAT' 'ULON' 'aice_m' 'aicen_m' 'alidf_ai_m' 'alidr_ai_m' 'alvdf_ai_m' 'alvdr_ai_m' 'blkmask' 'congel_m' 'divu_m' 'dxt' 'dxu' 'dyt' '...,['angle grid makes with latitude line on U grid' 'angle grid makes with latitude line on T grid' 'T cell width on East side' 'T cell width on North side' 'category maximum thickness' 'T grid ce...,['' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' ''],['' '' '' '' '' '' '' 'time: mean' '' '' 'time: mean' 'time: mean' 'time: mean' 'time: mean' 'time: mean' 'time: mean' '' 'time: mean' 'time: mean' '' '' '' '' 'time: mean' 'time: mean' 'time: m...,['radians' 'radians' 'm' 'm' 'm' 'degrees_north' 'degrees_east' 'C' 'degrees_north' 'degrees_east' '1' '1' '%' '%' '%' '%' '' 'cm/day' '%/day' 'm' 'm' 'm' 'm' 'W/m^2' 'W/m^2' 'W/m^2' 'cm/day' 'd...,seaIce
1,iceh.1900-02.nc,/g/data/cj50/access-om2/raw-output/access-om2-01/01deg_jra55v13_ryf9091/output000/ice/OUTPUT/iceh.1900-02.nc,seaIce.1mon.d2:2.nc:5.ni:3600.nj:2700,1mon,"1900-02-01, 00:00:00","1900-03-01, 00:00:00",['ANGLE' 'ANGLET' 'HTE' 'HTN' 'NCAT' 'TLAT' 'TLON' 'Tsfc_m' 'ULAT' 'ULON' 'aice_m' 'aicen_m' 'alidf_ai_m' 'alidr_ai_m' 'alvdf_ai_m' 'alvdr_ai_m' 'blkmask' 'congel_m' 'divu_m' 'dxt' 'dxu' 'dyt' '...,['angle grid makes with latitude line on U grid' 'angle grid makes with latitude line on T grid' 'T cell width on East side' 'T cell width on North side' 'category maximum thickness' 'T grid ce...,['' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' ''],['' '' '' '' '' '' '' 'time: mean' '' '' 'time: mean' 'time: mean' 'time: mean' 'time: mean' 'time: mean' 'time: mean' '' 'time: mean' 'time: mean' '' '' '' '' 'time: mean' 'time: mean' 'time: m...,['radians' 'radians' 'm' 'm' 'm' 'degrees_north' 'degrees_east' 'C' 'degrees_north' 'degrees_east' '1' '1' '%' '%' '%' '%' '' 'cm/day' '%/day' 'm' 'm' 'm' 'm' 'W/m^2' 'W/m^2' 'W/m^2' 'cm/day' 'd...,seaIce
2,iceh.1900-03.nc,/g/data/cj50/access-om2/raw-output/access-om2-01/01deg_jra55v13_ryf9091/output000/ice/OUTPUT/iceh.1900-03.nc,seaIce.1mon.d2:2.nc:5.ni:3600.nj:2700,1mon,"1900-03-01, 00:00:00","1900-04-01, 00:00:00",['ANGLE' 'ANGLET' 'HTE' 'HTN' 'NCAT' 'TLAT' 'TLON' 'Tsfc_m' 'ULAT' 'ULON' 'aice_m' 'aicen_m' 'alidf_ai_m' 'alidr_ai_m' 'alvdf_ai_m' 'alvdr_ai_m' 'blkmask' 'congel_m' 'divu_m' 'dxt' 'dxu' 'dyt' '...,['angle grid makes with latitude line on U grid' 'angle grid makes with latitude line on T grid' 'T cell width on East side' 'T cell width on North side' 'category maximum thickness' 'T grid ce...,['' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' ''],['' '' '' '' '' '' '' 'time: mean' '' '' 'time: mean' 'time: mean' 'time: mean' 'time: mean' 'time: mean' 'time: mean' '' 'time: mean' 'time: mean' '' '' '' '' 'time: mean' 'time: mean' 'time: m...,['radians' 'radians' 'm' 'm' 'm' 'degrees_north' 'degrees_east' 'C' 'degrees_north' 'degrees_east' '1' '1' '%' '%' '%' '%' '' 'cm/day' '%/day' 'm' 'm' 'm' 'm' 'W/m^2' 'W/m^2' 'W/m^2' 'cm/day' 'd...,seaIce
3,ocean.nc,/g/data/cj50/access-om2/raw-output/access-om2-01/01deg_jra55v13_ryf9091/output000/ocean/ocean.nc,ocean.3mon.grid_xt_ocean:3600.grid_xu_ocean:3600.grid_yt_ocean:2700.grid_yu_ocean:2700.neutral:80.neutralrho_edges:81.nv:2.potrho:80.potrho_edges:81.st_edges_ocean:76.st_ocean:75.sw_edges_ocean:76...,3m

## 5. Finding all variables in an experiment

You can get a list of all available variable names from an experiment with:


In [22]:
variables = catalog.search(name=experiment).unique().variable
print(variables)

['ty_trans_rho', 'vvel_m', 'area_t', 'ULON', 'passive_prydz', 'fsalt_ai_m', 'vatm_m', 'alvdr_ai_m', 'opening_m', 'time_bounds', 'tx_trans_rho', 'temp_global_ave', 'total_ocean_calving_heat', 'strairy_m', 'yt_ocean', 'tmask', 'wfimelt', 'temp_advection', 'uarea', 'frazil_m', 'pot_rho_1', 'strength_m', 'pot_temp', 'passive_ross', 'u_dot_grad_vert_pv', 'dyt', 'aicen_m', 'temp', 'geolon_t', 'uhrho_et', 'total_ocean_fprec', 'tx_trans_submeso', 'evap_heat', 'lw_heat', 'dxu', 'total_ocean_calving_melt_heat', 'TLON', 'total_ocean_lw_heat', 'geolon_c', 'total_ocean_runoff_heat', 'scalar_axis', 'xt_ocean_sub02', 'pot_rho_0', 'TLAT', 'sfc_hflux_coupler', 'alvdf_ai_m', 'sfc_hflux_from_runoff', 'NCAT', 'hs_m', 'total_ocean_melt', 'temp_rivermix', 'potrho_edges', 'time', 'total_ocean_hflux_evap', 'net_sfc_heating', 'total_ocean_runoff', 'Tsfc_m', 'sst_m', 'mld', 'usurf', 'salt', 'tau_y', 'tau_x', 'fmelttn_ai_m', 'frazil_3d_int_z', 'yu_ocean', 'ANGLE', 'ANGLET', 'bih_fric_v', 'grid_xu_ocean', 'fprec'

### We could similarly filter for any of the keys in our catalog - see the intake dataframe catalog below


In [23]:
catalog.search(name=experiment)

,model,description,realm,frequency,variable
name,,,,,
01deg_jra55v13_ryf9091,{ACCESS-OM2-01},{0.1 degree ACCESS-OM2 global model configuration with JRA55-do v1.3 RYF9091 repeat year forcing (May 1990 to Apr 1991)},"{ocean, seaIce}","{3mon, fx, 1mon, 1day, 3hr}","{ty_trans_rho, vvel_m, area_t, ULON, passive_prydz, fsalt_ai_m, vatm_m, alvdr_ai_m, opening_m, time_bounds, tx_trans_rho, temp_global_ave, total_ocean_calving_heat, strairy_m, yt_ocean, tmask, wfi..."


In [24]:
catalog.search(name=experiment, realm = 'ocean')

,model,description,realm,frequency,variable
name,,,,,
01deg_jra55v13_ryf9091,{ACCESS-OM2-01},{0.1 degree ACCESS-OM2 global model configuration with JRA55-do v1.3 RYF9091 repeat year forcing (May 1990 to Apr 1991)},{ocean},"{3mon, fx, 1mon, 1day, 3hr}","{ty_trans_rho, area_t, passive_prydz, time_bounds, tx_trans_rho, temp_global_ave, total_ocean_calving_heat, yt_ocean, wfimelt, temp_advection, pot_rho_1, pot_temp, passive_ross, u_dot_grad_vert_pv..."


In [25]:
# Lets pull out all the unique frequencies, just like we did for variable above
catalog.search(name=experiment, realm='ocean').unique().frequency

['3mon', 'fx', '1mon', '1day', '3hr']

We could also open the experiment (using square brackets) to search for variables, frequencies, etc. - but this opens the datastore: see how the output of the cell below is displayed differently.

If we open the datastore: 
1. It is slower - opening datastores requires extra work
2. The items we can search on might change - the datastore below contains no model field, for example.

The opened datastore can contain extra information, eg. `variable_long_name` below - so sometimes you might want to open it to search the datastore. In general, try to use `catalog.search(name='xyz',...)` before you use `catalog['xyz'].search(...)`, though.

In [26]:
catalog[experiment]

,unique
filename,3469
path,11947
file_id,22
frequency,5
start_date,3361
end_date,3360
variable,205
variable_long_name,197
variable_standard_name,36
variable_cell_methods,3


For more information about the available variables, you can use the following command function - just copy and paste it in where you need it:

In [1]:
from intake_esm.utils import MinimalExploder

def get_detailed_variable_info(intake_catalog, experiment_name : str, variable : str | None = None) -> "pd.Dataframe":
    """
    Get detailed information about all the variables available in an experiment contained within the catalog.

    If a specific variable is passed, then the returned dataframe will be filtered to include only information
    about that variable

    Returns a pandas dataframe, reorganised to use the variable as the index.

    Parameters:
    -----------
    intake_catalog: 
        The variable holding the intake catalog. If you have opened the catalog using
        `cat = intake.cat.access_nri`, then `intake_catalog=cat`, etc.
    experiment_name: str
        The name of the experiment you are interested in. Eg. `experiment = "01deg_jra55v13_ryf9091"`
    variable: str | None
        If you want detailed information about just a single variable, then pass it here. For 
        example, if you only want information about potential temperature, pass `variable='pot_temp'`
    """


    expt_ds = intake_catalog[experiment_name]
    df = MinimalExploder(expt_ds.esmcat.pl_df)()

    df = df.unique('variable').sort('variable')
        
    df = df.to_pandas().set_index("variable")

    return df
    

To get detailed info about all variables:

In [28]:
df = get_detailed_variable_info(catalog, experiment)
df.head(10)

,filename,path,file_id,frequency,start_date,end_date,variable_long_name,variable_standard_name,variable_cell_methods,variable_units,realm
variable,,,,,,,,,,,
ANGLE,iceh.1900-01.nc,/g/data/cj50/access-om2/raw-output/access-om2-01/01deg_jra55v13_ryf9091/output000/ice/OUTPUT/iceh.1900-01.nc,seaIce.1mon.d2:2.nc:5.ni:3600.nj:2700,1mon,"1900-01-01, 00:00:00","1900-02-01, 00:00:00",angle grid makes with latitude line on U grid,,,radians,seaIce
ANGLET,iceh.1900-01.nc,/g/data/cj50/access-om2/raw-output/access-om2-01/01deg_jra55v13_ryf9091/output000/ice/OUTPUT/iceh.1900-01.nc,seaIce.1mon.d2:2.nc:5.ni:3600.nj:2700,1mon,"1900-01-01, 00:00:00","1900-02-01, 00:00:00",angle grid makes with latitude line on T grid,,,radians,seaIce
HTE,iceh.1900-01.nc,/g/data/cj50/access-om2/raw-output/access-om2-01/01deg_jra55v13_ryf9091/output000/ice/OUTPUT/iceh.1900-01.nc,seaIce.1mon.d2:2.nc:5.ni:3600.nj:2700,1mon,"1900-01-01, 00:00:00","1900-02-01, 00:00:00",T cell width on East side,,,m,seaIce
HTN,iceh.1900-01.nc,/g/data/cj50/access-om2/raw-output/access-om2-01/01deg_jra55v13_ryf9091/output000/ice/OUTPUT/iceh.1900-01.nc,seaIce.1mon.d2:2.nc:5.ni:3600.nj:2700,1mon,"1900-01-01, 00:00:00","1900-02-01, 00:00:00",T cell width on North side,,,m,seaIce
NCAT,iceh.1900-01.nc,/g/data/cj50/access-om2/raw-output/access-om2-01/01deg_jra55v13_ryf9091/output000/ice/OUTPUT/iceh.1900-01.nc,seaIce.1mon.d2:2.nc:5.ni:3600.nj:2700,1mon,"1900-01-01, 00:00:00","1900-02-01, 00:00:00",category maximum thickness,,,m,seaIce
TLAT,iceh.1900-01.nc,/g/data/cj50/access-om2/raw-output/access-om2-01/01deg_jra55v13_ryf9091/output000/ice/OUTPUT/iceh.1900-01.nc,seaIce.1mon.d2:2.nc:5.ni:3600.nj:2700,1mon,"1900-01-01, 00:00:00","1900-02-01, 00:00:00",T grid center latitude,,,degrees_north,seaIce
TLON,iceh.1900-01.nc,/g/data/cj50/access-om2/raw-output/access-om2-01/01deg_jra55v13_ryf9091/output000/ice/OUTPUT/iceh.1900-01.nc,seaIce.1mon.d2:2.nc:5.ni:3600.nj:2700,1mon,"1900-01-01, 00:00:00","1900-02-01, 00:00:00",T grid center longitude,,,degrees_east,seaIce
Tsfc_m,iceh.1900-01.nc,/g/data/cj50/access-om2/raw-output/access-om2-01/01deg_jra55v13_ryf9091/output000/ice/OUTPUT/iceh.1900-01.nc,seaIce.1mon.d2:2.nc:5.ni:3600.nj:2700,1mon,"1900-01-01, 00:00:00","1900-02-01, 00:00:00",snow/ice surface temperature,,time: mean,C,seaIce
ULAT,iceh.1900-01.nc,/g/data/cj50/access-om2/raw-output/access-om2-01/01deg_jra55v13_ryf9091/output000/ice/OUTPUT/iceh.1900-01.nc,seaIce.1mon.d2:2.nc:5.ni:3600.nj:2700,1mon,"1900-01-01, 00:00:00","1900-02-01, 00:00:00",U grid center latitude,,,degrees_north,seaIce


Say we are only interested in zonal wind stress, `tau_x`:

In [29]:
df = get_detailed_variable_info(catalog, experiment, 'tau_x')
df.head(10)

,filename,path,file_id,frequency,start_date,end_date,variable_long_name,variable_standard_name,variable_cell_methods,variable_units,realm
variable,,,,,,,,,,,
ANGLE,iceh.1900-01.nc,/g/data/cj50/access-om2/raw-output/access-om2-01/01deg_jra55v13_ryf9091/output000/ice/OUTPUT/iceh.1900-01.nc,seaIce.1mon.d2:2.nc:5.ni:3600.nj:2700,1mon,"1900-01-01, 00:00:00","1900-02-01, 00:00:00",angle grid makes with latitude line on U grid,,,radians,seaIce
ANGLET,iceh.1900-01.nc,/g/data/cj50/access-om2/raw-output/access-om2-01/01deg_jra55v13_ryf9091/output000/ice/OUTPUT/iceh.1900-01.nc,seaIce.1mon.d2:2.nc:5.ni:3600.nj:2700,1mon,"1900-01-01, 00:00:00","1900-02-01, 00:00:00",angle grid makes with latitude line on T grid,,,radians,seaIce
HTE,iceh.1900-01.nc,/g/data/cj50/access-om2/raw-output/access-om2-01/01deg_jra55v13_ryf9091/output000/ice/OUTPUT/iceh.1900-01.nc,seaIce.1mon.d2:2.nc:5.ni:3600.nj:2700,1mon,"1900-01-01, 00:00:00","1900-02-01, 00:00:00",T cell width on East side,,,m,seaIce
HTN,iceh.1900-01.nc,/g/data/cj50/access-om2/raw-output/access-om2-01/01deg_jra55v13_ryf9091/output000/ice/OUTPUT/iceh.1900-01.nc,seaIce.1mon.d2:2.nc:5.ni:3600.nj:2700,1mon,"1900-01-01, 00:00:00","1900-02-01, 00:00:00",T cell width on North side,,,m,seaIce
NCAT,iceh.1900-01.nc,/g/data/cj50/access-om2/raw-output/access-om2-01/01deg_jra55v13_ryf9091/output000/ice/OUTPUT/iceh.1900-01.nc,seaIce.1mon.d2:2.nc:5.ni:3600.nj:2700,1mon,"1900-01-01, 00:00:00","1900-02-01, 00:00:00",category maximum thickness,,,m,seaIce
TLAT,iceh.1900-01.nc,/g/data/cj50/access-om2/raw-output/access-om2-01/01deg_jra55v13_ryf9091/output000/ice/OUTPUT/iceh.1900-01.nc,seaIce.1mon.d2:2.nc:5.ni:3600.nj:2700,1mon,"1900-01-01, 00:00:00","1900-02-01, 00:00:00",T grid center latitude,,,degrees_north,seaIce
TLON,iceh.1900-01.nc,/g/data/cj50/access-om2/raw-output/access-om2-01/01deg_jra55v13_ryf9091/output000/ice/OUTPUT/iceh.1900-01.nc,seaIce.1mon.d2:2.nc:5.ni:3600.nj:2700,1mon,"1900-01-01, 00:00:00","1900-02-01, 00:00:00",T grid center longitude,,,degrees_east,seaIce
Tsfc_m,iceh.1900-01.nc,/g/data/cj50/access-om2/raw-output/access-om2-01/01deg_jra55v13_ryf9091/output000/ice/OUTPUT/iceh.1900-01.nc,seaIce.1mon.d2:2.nc:5.ni:3600.nj:2700,1mon,"1900-01-01, 00:00:00","1900-02-01, 00:00:00",snow/ice surface temperature,,time: mean,C,seaIce
ULAT,iceh.1900-01.nc,/g/data/cj50/access-om2/raw-output/access-om2-01/01deg_jra55v13_ryf9091/output000/ice/OUTPUT/iceh.1900-01.nc,seaIce.1mon.d2:2.nc:5.ni:3600.nj:2700,1mon,"1900-01-01, 00:00:00","1900-02-01, 00:00:00",U grid center latitude,,,degrees_north,seaIce


If you have any further questions after reading this notebook and the documentation linked from this notebook, please open an issue in the [ACCESS-NRI Intake catalog Github repo](https://github.com/ACCESS-NRI/access-nri-intake-catalog) or open topic on the [ACCESS-Hive forum](https://forum.access-hive.org.au/).

In [30]:
client.close()